## Read dataset

In [1]:
# import libraries
from sql_functions import *
from pandas import DataFrame
import pickle
import nltk
import re

# query data
columns = ["shortcode","caption","label"]
data = query_all(columns,exclude_none="caption")
data.head(10)

# hitung jumlah baris
na = data['label'].isna().sum()
print("Jumlah Data:",data.shape[0])
print("Jumlah NA:",na)
print("Data Berlabel:",data.shape[0]-na)

Jumlah Data: 1545
Jumlah NA: 0
Data Berlabel: 1545


## Do text preprocessing

In [2]:
nltk.download("stopwords")
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to C:\Users\Ekky
[nltk_data]     Armandi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
def stopwords_removal(word):
    filtering = stopwords.words("indonesian","english")
    fit = list(filter(lambda x: False if x in filtering else True, word))
    return fit
    
data['tokenized'] = data['caption'].apply(lambda caption: re.findall("\w+",caption.lower()))
data['tokenized'] = data['tokenized'].apply(stopwords_removal)
data['tokenized'] = data['tokenized'].apply(lambda words: " ".join(words))

data.head(10)

,shortcode,caption,label,tokenized
0,CdkKXK9p7hOdKxgtry4sggDIJtX94oAjuaEvz00,Nama barang : sepatu size 37\nKondisi : new bl...,AVAIL,nama barang sepatu size 37 kondisi new blm dip...
1,CdkJy3Mp6N68qiy4Vk4vpat0dolAzLmdyxViMw0,⛔️SUDAH TERJUAL⛔️ \n\nNama barang : tas\nKondi...,SOLD,terjual nama barang tas kondisi new blm dipaka...
2,CdkJlJmpzRLEaFIxrn3FMQprY5ulloR_lnpC_40,Nama barang : VANS SLIP ON BLACK CLASSIC 7 us ...,AVAIL,nama barang vans slip on black classic 7 us 39...
3,CdkIIy3JjxkwSGSppwD-g4r21Ws80rAeNSXFYU0,⛔️SUDAH TERJUAL⛔️ \n\nDijual : Tiket nonton KK...,SOLD,terjual dijual tiket nonton kkn desa penari un...
4,Cdj-myspRqPPpfoGbjGV8RmyYgiNVrehExaVo40,⛔️SUDAH TERJUAL⛔️ \n\nNama barang: Pen tablet ...,SOLD,terjual nama barang pen tablet drawing pad hui...
5,Cdj-Qtwp6hQe589X95b9akSYwQZKN4dIeH-ArQ0,1. Nama barang : Bruklat hitam fit to L\nKondi...,AVAIL,1 nama barang bruklat hitam fit to l kondisi b...
6,Cdj99kDJ1EIscvToboJ2lR3cOVz6_PPgNhbhCY0,Nama barang : Sepatu Heels Payless VE Janine\n...,AVAIL,nama barang sepatu heels payless ve janine war...
7,Cdj6_M0JF6eVSkOcLvfUeMZvwW20NUG0qORlGY0,Nama Barang : ip xr 64gb red\nKondisi : mulus ...,AVAIL,nama barang ip xr 64gb red kondisi mulus nomin...
8,Cdj6lM3JfycyZuqXvsQrgTrgXw3uu0Y6d0jW_00,1. Nama barang : instax square 6\nKondisi : d...,AVAIL,1 nama barang instax square 6 kondisi dibeli 2...
9,CdiqfyCJ1ZqtiPiR-5-rgS4_AD80cFVPPDRnOk0,⛔️SUDAH TERJUAL⛔️ \n\nNama barang : Nama Baran...,SOLD,terjual nama barang nama barang iphone 8 64gb ...


In [4]:
barkas = data[['shortcode','tokenized','label']]
barkas = barkas.set_index('shortcode')
barkas = barkas.astype({'tokenized': 'string', 'label': 'category'})
barkas.columns = ['caption','label']
no_label = barkas[barkas['label'].isna()]
barkas = barkas.dropna(subset="label")

print("Barkas Dataset")
display(barkas.head(10))
print(barkas.shape)

print("\nBarkas Data Types")
barkas.dtypes

Barkas Dataset


,caption,label
shortcode,,
CdkKXK9p7hOdKxgtry4sggDIJtX94oAjuaEvz00,nama barang sepatu size 37 kondisi new blm dip...,AVAIL
CdkJy3Mp6N68qiy4Vk4vpat0dolAzLmdyxViMw0,terjual nama barang tas kondisi new blm dipaka...,SOLD
CdkJlJmpzRLEaFIxrn3FMQprY5ulloR_lnpC_40,nama barang vans slip on black classic 7 us 39...,AVAIL
CdkIIy3JjxkwSGSppwD-g4r21Ws80rAeNSXFYU0,terjual dijual tiket nonton kkn desa penari un...,SOLD
Cdj-myspRqPPpfoGbjGV8RmyYgiNVrehExaVo40,terjual nama barang pen tablet drawing pad hui...,SOLD
Cdj-Qtwp6hQe589X95b9akSYwQZKN4dIeH-ArQ0,1 nama barang bruklat hitam fit to l kondisi b...,AVAIL
Cdj99kDJ1EIscvToboJ2lR3cOVz6_PPgNhbhCY0,nama barang sepatu heels payless ve janine war...,AVAIL
Cdj6_M0JF6eVSkOcLvfUeMZvwW20NUG0qORlGY0,nama barang ip xr 64gb red kondisi mulus nomin...,AVAIL
Cdj6lM3JfycyZuqXvsQrgTrgXw3uu0Y6d0jW_00,1 nama barang instax square 6 kondisi dibeli 2...,AVAIL


(1545, 2)

Barkas Data Types


caption      string
label      category
dtype: object

In [5]:
# import ML libraries
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

model = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', MultinomialNB())
])

## Split Dataset

In [6]:
X = barkas['caption']
y = barkas['label']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=300)

print("X_train shape",X_train.shape)
print("X_test shape",X_test.shape)

X_train shape (1236,)
X_test shape (309,)


## Train the dataset

In [7]:
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
acr = accuracy_score(y_test,y_pred)

print(f"Tingkat akurasi model: {100*acr:.2f}%")

Tingkat akurasi model: 79.61%


In [ ]:
# no_label['predicted'] = model.predict(no_label['caption'])
# no_label

## Export the model

In [9]:
pickle.dump(model,open("NB_model.pkl","wb"))